<a href="https://colab.research.google.com/github/ArunGoddati/AJACKUS_Assignment/blob/main/TaskForge_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import uuid
from datetime import datetime

class Task:
    def __init__(self, title, priority, due_date, status="Pending", task_id=None):
        self.id = task_id if task_id else str(uuid.uuid4())[:8]
        self.title = title
        self.priority = priority
        self.due_date = due_date
        self.status = status

    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "priority": self.priority,
            "due_date": self.due_date,
            "status": self.status
        }

    @classmethod
    def from_dict(cls, data):
        return cls(
            title=data["title"],
            priority=data["priority"],
            due_date=data["due_date"],
            status=data["status"],
            task_id=data["id"]
        )


In [3]:
import json
from datetime import timedelta

class TaskManager:
    def __init__(self):
        self.task_list = []

    def add_task(self, title, priority, due_date):
        task = Task(title, priority, due_date)
        self.task_list.append(task)

    def view_tasks(self, filter_by=None, value=None):
        tasks = self.task_list

        if filter_by == "status":
            tasks = [t for t in tasks if t.status.lower() == value.lower()]
        elif filter_by == "due_date":
            today = datetime.today().date()
            if value == "today":
                tasks = [t for t in tasks if t.due_date == today.isoformat()]
            elif value == "week":
                week_later = today + timedelta(days=7)
                tasks = [t for t in tasks if today.isoformat() <= t.due_date <= week_later.isoformat()]

        if not tasks:
            print("No tasks found.")
        else:
            for task in tasks:
                print(f"{task.id} | {task.title} | {task.priority} | {task.due_date} | {task.status}")

    def update_task(self, task_id, new_title=None, new_priority=None, new_due_date=None):
        for task in self.task_list:
            if task.id == task_id:
                task.title = new_title or task.title
                task.priority = new_priority or task.priority
                task.due_date = new_due_date or task.due_date
                return True
        return False

    def mark_complete(self, task_id):
        for task in self.task_list:
            if task.id == task_id:
                task.status = "Completed"
                return True
        return False

    def delete_task(self, task_id):
        original_len = len(self.task_list)
        self.task_list = [t for t in self.task_list if t.id != task_id]
        return len(self.task_list) < original_len

    def save_to_file(self, filename="tasks.json"):
        with open(filename, "w") as f:
            json.dump([task.to_dict() for task in self.task_list], f, indent=2)

    def load_from_file(self, filename="tasks.json"):
        try:
            with open(filename, "r") as f:
                content = f.read().strip()
                if not content:
                    self.task_list = []
                    return
                tasks = json.loads(content)
                self.task_list = [Task.from_dict(t) for t in tasks]
        except (FileNotFoundError, json.JSONDecodeError):
            self.task_list = []


In [4]:
tm = TaskManager()

tm.add_task("Finish resume", "High", "2025-08-07")
tm.add_task("Buy groceries", "Medium", "2025-08-04")

tm.view_tasks()

task_id = tm.task_list[0].id
tm.update_task(task_id, new_title="Finish CV and resume")

tm.mark_complete(task_id)

tm.delete_task(task_id)

tm.save_to_file()
tm.load_from_file()

tm.view_tasks()


b284236b | Finish resume | High | 2025-08-07 | Pending
dde973e0 | Buy groceries | Medium | 2025-08-04 | Pending
dde973e0 | Buy groceries | Medium | 2025-08-04 | Pending
